# Загрузка Pandas и очистка данных

In [40]:
# загружаю необходимые библиотеки
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime, timedelta
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

from pandas import Series, DataFrame
%matplotlib inline
def get_rest_count(value):
    return rest_count[value]


In [2]:
# создаю словарь со странами и столицами
city_country = {'London': 'UK', 'Paris': 'France', 'Madrid': 'Spain', 'Barcelona': 'Spain',
                'Berlin': 'Germany', 'Milan': 'Italy', 'Rome': 'Italy', 'Prague': 'Czech',
                'Lisbon': 'Portugalia', 'Vienna': 'Austria', 'Amsterdam': 'Nederlands', 'Brussels': 'Belgium',
                'Hamburg': 'Germany', 'Munich': 'Germany', 'Lyon': 'France', 'Stockholm': 'Sweden',
                'Budapest': 'Hungary', 'Warsaw': 'Poland', 'Dublin': 'Ireland', 'Copenhagen': 'Denmark',
                'Athens': 'Greece', 'Edinburgh': 'Schotland', 'Zurich': 'Switzerland', 'Oporto': 'Portugalia',
                'Geneva': 'Switzerland', 'Krakow': 'Poland', 'Oslo': 'Norway', 'Helsinki': 'Finland',
                'Bratislava': 'Slovakia', 'Luxembourg': 'Luxembourg', 'Ljubljana': 'Slovenija'}

In [3]:
# создаю словарь, является ли город столицей
city_capital = {'London': 1, 'Paris': 1, 'Madrid': 1, 'Barcelona': 0, 'Berlin': 1,
                'Milan': 0, 'Rome': 1, 'Prague': 1, 'Lisbon': 1, 'Vienna': 1,
                'Amsterdam': 1, 'Brussels': 1, 'Hamburg': 0, 'Munich': 0, 'Lyon': 0,
                'Stockholm': 1, 'Budapest': 1, 'Warsaw': 1, 'Dublin': 1, 'Copenhagen': 1,
                'Athens': 1, 'Edinburgh': 1, 'Zurich': 1, 'Oporto': 0, 'Geneva': 1,
                'Krakow': 1, 'Oslo': 1, 'Helsinki': 1, 'Bratislava': 1, 'Luxembourg': 1, 'Ljubljana': 1}

In [4]:
# создаю словарь с информацией о населении города
city_population = {'London': 8173900, 'Paris': 2240621, 'Madrid': 3155360, 'Barcelona': 1593075,
                   'Berlin': 3326002, 'Milan': 1331586, 'Rome': 2870493, 'Prague': 1272690,
                   'Lisbon': 547733, 'Vienna': 1765649, 'Amsterdam': 825080, 'Brussels': 144784,
                   'Hamburg': 1718187, 'Munich': 1364920, 'Lyon': 496343, 'Stockholm': 1981263,
                   'Budapest': 1744665, 'Warsaw': 1720398, 'Dublin': 506211, 'Copenhagen': 1246611,
                   'Athens': 3168846, 'Edinburgh': 476100, 'Zurich': 402275, 'Oporto': 221800,
                   'Geneva': 196150, 'Krakow': 756183, 'Oslo': 673469, 'Helsinki': 574579,
                   'Bratislava': 413192, 'Luxembourg': 576249, 'Ljubljana': 277554}

In [1]:
# создаю словарь с указанием количества ресторанов для каждого города
res_count = {'Paris': 17593, 'Stockholm': 3131, 'London': 22366, 'Berlin': 8110,
             'Munich': 3367, 'Oporto': 2060, 'Milan': 7940, 'Bratislava': 1331,
             'Vienna': 4387, 'Rome': 12086, 'Barcelona': 10086, 'Madrid': 11562,
             'Dublin': 2706, 'Brussels': 3703, 'Zurich': 1901, 'Warsaw': 3210,
             'Budapest': 3445, 'Copenhagen': 2637, 'Amsterdam': 4189, 'Lyon': 2833,
             'Hamburg': 3501, 'Lisbon': 4985, 'Prague': 5850, 'Oslo': 1441,
             'Helsinki': 1661, 'Edinburgh': 2248, 'Geneva': 1753, 'Ljubljana': 647,
             'Athens': 2814, 'Luxembourg': 759, 'Krakow': 1832}

In [6]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [7]:
df = pd.read_csv('main_task.csv')
# сделаем названия столбцов подходящими для дальнешей работы

df.columns = ['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
              'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA']
df.columns = map(lambda name: name.lower(), df.columns)
df.columns = df.columns.str.replace(' ', '_')

In [8]:
# Относительный ранг, чем больше к единице, тем выше ранг ресторана
rest_count = df.city.value_counts()


def get_rest_count(value):
    return rest_count[value]


df['relative_ranking'] = 1 - \
    (df['ranking'] / df['city'].map(df.groupby(['city'])['ranking'].max()))


# Добавляю столбец со страной
df['country'] = df['city'].map(city_country)

# Создаю признак, отражающий количество ресторанов в городе, в котором расположен данный ресторан
df['restaurants_count'] = df['city'].map(res_count)


# Создаю признак с населением города
df['population'] = df['city'].map(city_population)

# признак пропорция кол-ва ресторанов и населения
df['per_capital'] = df['population']/df['restaurants_count']

# Состоит ли ресторан в сети
restaurant_chain = set()
for chain in df['restaurant_id']:
    restaurant_chain.update(chain)


def find_item(cell):
    if item in cell:
        return 1
    return 0


for item in restaurant_chain:
    df['restaurant_chain'] = df['restaurant_id'].apply(find_item)

# Сколько городов представлено в наборе данных?
len(df['city'].unique())

31

In [9]:
# CUISINES

# столбец с кухнями разделяем на отдельные виды
df['cuisine_style'] = df.cuisine_style.str.replace(
    ']', '').str.replace('[', '')
o = df[['id_ta', 'cuisine_style']]
o.cuisine_style.fillna("'Europian'", inplace=True)
s = o['cuisine_style'].str.split(',').apply(Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'cuisine_style'
del o['cuisine_style']
k = o.join(s)

# Создаем признак "Кол-во кухонь в каждом ресторане"
table_with_num = k.copy()
table_with_num['num_of_cuis'] = (table_with_num.groupby(
    ['id_ta'])['cuisine_style']).transform('nunique')
table_with_num.drop(['cuisine_style'], axis=1, inplace=True)
table_with_num = table_with_num.drop_duplicates()
df = df.merge(table_with_num, how='left')

# среднее количество кухонь на ресторан в городе
df['cuisine_count_mean'] = df['city'].map(
    df.groupby('city')['num_of_cuis'].mean())

# Добавляю признак о наличии веганской кухни в ресторане

df['veg'] = df['cuisine_style'].str.contains(r'Veg', na=True)
df['veg'] = df['veg'].astype(int)
CuisinesDict = {}
# Функция заполнения словаря CuisinesDict


def CuisinesDictFill(cuisines):
    global CuisinesDict
    # из входящей строки вида "['German', 'Central European', 'Vegetarian Friendly']"
    cuisines = str(cuisines).replace("['", "").replace(
        "']", "").replace("', '", ",").split(',')
    # получили список отдельных значений вида ['German', 'Central European', 'Vegetarian Friendly']
    if cuisines[0] != 'nan':  # если список кухонь не пустой
        for i in cuisines:  # перебор полученного списка
            if CuisinesDict.get(i) == None:  # нет такой кухни в словаре CuisinesDict
                # добавить кухню (ключ) в CuisinesDict со значением 1
                CuisinesDict[i] = 1
            else:
                # увеличить значение для этой кухни (ключа) в словаре CuisinesDict
                CuisinesDict[i] += 1


df['cuisine_style'].apply(CuisinesDictFill)

# 4.3.3 среднее кол-во кухонь в одном ресторане
k.groupby('id_ta')['cuisine_style'].nunique().sort_values(
    ascending=False).replace(0, 1).mean()

# 4.3.2 какая кухня представлена в наибольшем кол-ве ресторанов
k.groupby(['cuisine_style'])['id_ta'].nunique(
).sort_values(ascending=False).head(1)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


cuisine_style
 'Vegetarian Friendly'    10886
Name: id_ta, dtype: int64

In [10]:
# 2.REVIEWS


# Разделим столбец с отзывами на текст и дату

df['reviews'] = df.reviews.str.replace(']', '').str.replace('[', '')
# 4.4.1. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
df['reviews'][1]
reviews_table = df['reviews'].str.split(', ', expand=True)
reviews_table[3].dropna(inplace=True)
df['rew_date'] = pd.to_datetime(reviews_table[3], errors='coerce')
df['rew_date1'] = pd.to_datetime(reviews_table[2], errors='coerce')
df['diff'] = df['rew_date1']-df['rew_date']
df['rew_date'] = pd.to_datetime(df['rew_date'], format='%Y-%m-%d')
df['rew_date1'] = pd.to_datetime(df['rew_date1'], format='%Y-%m-%d')
df['diff'] = (df.rew_date1 - df.rew_date).dt.days

# создаю столбец с разницей в отзывах, но поддаю сомнению слишком большую разницу в днях,
# и решаю взять наиболее свежие отзывы
# убираю выбросы

daty = pd.DataFrame({'DATE': [pd.Timestamp(x) for x in df['rew_date']]})
# print(daty)
qa = daty['DATE'].quantile(0.1)  # lower 10%
qb = daty['DATE'].quantile(0.9)  # higher 10%
# remove outliers
df['new_date'] = daty[(daty['DATE'] >= qa) & (daty['DATE'] <= qb)]


da = pd.DataFrame({'DATES': [pd.Timestamp(x) for x in df['rew_date1']]})
# print(daty)
qa = da['DATES'].quantile(0.1)  # lower 10%
qb = da['DATES'].quantile(0.9)  # higher 10%

# Получаю новый признака разницы дат отзывов
df['new_date1'] = da[(da['DATES'] >= qa) & (da['DATES'] <= qb)]

df['new_date'] = pd.to_datetime(df['new_date'], format='%Y-%m-%d')
df['new_date1'] = pd.to_datetime(df['new_date1'], format='%Y-%m-%d')
df['new_diff'] = (df.new_date1 - df.new_date).dt.days

# Заменяю "новизну" отзывов на ранги
df['last_review'] = df.new_date1.dt.year
df['last_review'] = df['last_review'].replace(2017.0, 2)
df['last_review'] = df['last_review'].replace(2016.0, 1)
df['last_review'] = df['last_review'].replace(2018.0, 3)

# Создаю новый признак с разницей между актуальным днем и последним отзывом
df['fix'] = '2020-12-03 00:00'
df['fix'] = pd.DataFrame({'DATES': [pd.Timestamp(x) for x in df['fix']]})
df['last_'] = (df.fix - df.new_date1).dt.days

In [17]:
# Признаки на основании текста отзывов

df['good_rev'] = df['reviews'].str.contains(r'good', na=True)  # ,
df['good_rev'] = df['good_rev'].astype(int)

df['best_rev'] = df['reviews'].str.contains(r'best', na=True)  # ,
df['best_rev'] = df['best_rev'].astype(int)


df['great_rev'] = df['reviews'].str.contains(r'great', na=True)  # ,
df['great_rev'] = df['great_rev'].astype(int)

df['bad_rev'] = df['reviews'].str.contains(r'bad', na=True)  # ,
df['bad_rev'] = df['bad_rev'].astype(int)


# Кол-во на отзывов на душу населения
df['reviews_per_people100'] = df.apply(lambda x:
                                       round(x['number_of_reviews']*10000/x['population'], 4), axis=1)

# 4.4.1. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
df['reviews'][1]
reviews_table = df['reviews'].str.split(', ', expand=True)
reviews_table[3].dropna(inplace=True)
df['rew_date'] = pd.to_datetime(reviews_table[3], errors='coerce')
df['rew_date1'] = pd.to_datetime(reviews_table[2], errors='coerce')
df['diff'] = df['rew_date1']-df['rew_date']
print((df['rew_date1']-df['rew_date']).max())
print(df['rew_date'].max())

3127 days 00:00:00
2018-02-26 00:00:00


In [11]:
# Заменю ценовую категорию на числовые значения
price_range_dict = {'$': 10, '$$ - $$$': 100, '$$$$': 1000}
df['price_range'] = df['price_range'].map(price_range_dict)

In [12]:
# Загружаю новый датасет о наличии мишленовской звезды у ресторанов по странам

mi = pd.read_csv('michelin_stars.csv')
mi = mi.drop(['the_geom', 'cartodb_id',
              'cartodb_georef_status', 'guide_year'], axis=1)
df = df.merge(mi, how='left')

In [13]:
# Загружаю новый датасет о 50 лучших ресторанах и доп информации о ресторанах
best = pd.read_csv('50_best_restaurants_database.csv')
best['name'] = best['Restaurant']
best.columns = map(lambda name: name.lower(), best.columns)
best.columns
b = best.drop(['ranking', 'restaurant', 'lat', 'lon',
               'stars', 'website', 'description'], axis=1)
df = df.merge(b, how='left')

In [14]:
# Загружаю новый датасет, чтобы взять название и id ресторана
ta = pd.read_csv('TA_restaurants_curated.csv')
ta.columns = map(lambda name: name.lower(), ta.columns)
ta.columns
ta = ta.drop(['unnamed: 0', 'city', 'cuisine style', 'ranking', 'rating',
              'price range', 'number of reviews', 'reviews', 'url_ta'], axis=1)
df = df.merge(ta, how='left')

In [15]:
# Создаю новый признак о средней цене дегустационного меню в ресторанах,
# заранее переведя все в USD, курс указан на начало декабря
df.currency.value_counts()
currency_dict = {'EUR': 1.21, 'GBP': 1.3481, 'DKK': 0.1629, 'SEK': 0.12}

df['currency_for_usd'] = df['currency'].map(currency_dict)
df['price_menu_in_usd'] = df['currency_for_usd']*df['menu']
df.currency_for_usd.value_counts()

1.2100    28377
1.3481    11514
0.1629     1318
0.1200      820
Name: currency_for_usd, dtype: int64

In [34]:
# проведем корреляционный анализ числовых признаков
df_num = df.copy()
df_num = df_num[['ranking', 'rating',
                 'price_range', 'number_of_reviews',
                 'relative_ranking', 'restaurants_count', 'population',
                 'per_capital', 'restaurant_chain', 'num_of_cuis', 'cuisine_count_mean',
                 'veg','new_diff', 'last_review', 'fix', 'last_',
                 'restaurants_with_three_stars', 'restaurants_with_two_stars',
                 'restaurants_with_one_star',  'price_menu_in_usd', 'good_rev', 'best_rev',
                 'great_rev', 'bad_rev', 'reviews_per_people100']]
df_num.corr()
# и далее рассмотрим по отдельности признаки, наиболее сильно влияющие на оценку по тесту

,ranking,rating,price_range,number_of_reviews,relative_ranking,restaurants_count,population,per_capital,restaurant_chain,num_of_cuis,...,last_,restaurants_with_three_stars,restaurants_with_two_stars,restaurants_with_one_star,price_menu_in_usd,good_rev,best_rev,great_rev,bad_rev,reviews_per_people100
ranking,1.000000,-0.417990,-0.102121,-0.263506,-0.662214,0.641344,0.447835,-0.160277,0.216675,-0.367357,...,0.192162,0.577124,0.554539,0.555432,-0.043256,-0.056769,-0.035048,-0.044312,0.032362,-0.220840
rating,-0.417990,1.000000,0.069885,0.030630,0.558912,-0.046973,-0.016680,0.041642,-0.141302,0.125374,...,-0.094989,-0.040254,-0.031965,-0.032390,-0.011087,-0.009445,0.055406,0.051515,-0.074569,0.025217
price_range,-0.102121,0.069885,1.000000,0.191245,0.160331,0.017139,-0.013986,-0.023135,-0.002276,0.073368,...,-0.082298,0.041745,0.041569,0.038553,0.039832,-0.011486,0.007314,-0.006154,-0.010250,0.083465
number_of_reviews,-0.263506,0.030630,0.191245,1.000000,0.404198,0.033033,0.023881,-0.029546,-0.047114,0.387145,...,-0.266263,0.004210,0.011709,0.025167,-0.022347,0.049086,0.004199,0.020003,-0.003303,0.560309
relative_ranking,-0.662214,0.558912,0.160331,0.404198,1.000000,0.002461,-0.000881,-0.001913,-0.187995,0.532278,...,-0.331335,0.007093,0.006781,0.006755,0.004275,0.086859,0.046570,0.073952,-0.052000,0.255349
restaurants_count,0.641344,-0.046973,0.017139,0.033033,0.002461,1.000000,0.742625,-0.208387,0.058212,-0.022174,...,-0.059105,0.882008,0.854294,0.865496,-0.149497,-0.004146,-0.009098,0.006240,-0.001846,-0.185709
population,0.447835,-0.016680,-0.013986,0.023881,-0.000881,0.742625,1.000000,0.347063,0.075113,0.079923,...,-0.101398,0.307150,0.295605,0.318152,-0.498948,0.003025,0.000465,0.013552,0.002968,-0.168873
per_capital,-0.160277,0.041642,-0.023135,-0.029546,-0.001913,-0.208387,0.347063,1.000000,0.040305,0.079381,...,-0.050777,-0.611410,-0.603278,-0.624582,-0.336303,0.010186,0.014373,0.006999,0.002044,-0.103815
restaurant_chain,0.216675,-0.141302,-0.002276,-0.047114,-0.187995,0.058212,0.075113,0.040305,1.000000,-0.064232,...,0.035696,0.044644,0.047700,0.037353,0.005717,-0.005906,-0.007306,-0.011263,0.009125,-0.035303
num_of_cuis,-0.367357,0.125374,0.073368,0.387145,0.532278,-0.022174,0.079923,0.079381,-0.064232,1.000000,...,-0.319559,-0.152331,-0.146661,-0.141414,-0.130691,0.083888,0.015506,0.043552,-0.010652,0.256712


In [37]:
# наиболее сильная корреляция наблюдается в restaurant_chain, num_of_cuis, ranking 

#Ranking сильно коррелирует с Rating, возможно потому что признание ресторана в городе
#безусловно будет влиять на рейтинг

#Также можно предположить что количетсво типов кухонь в ресторане хорошо влияет на рейтинг, т.к.
#иногда удобно наряду с сытными грузинскими блюдами заказать Healthy food из Veg меню

#Наличие ресторана в сети ресторанов, мне кажется, может сказываться на рейтинге положительно,
#в случае, если четко соблюдается регламент сети. Те же рестораны Fast food, в незнакомом городе
#увидев вывеску KFC, например, сразу вспоминаешь аппетитные куриные ножки и "мысленный" рейтинг ресторана
#увеличивается

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [28]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['restaurant_id', 'city', 'cuisine_style',
             'reviews', 'rating', 'rew_date', 'rew_date1', 'fix', 'diff', 'new_date', 'new_date1', 'url_ta', 'id_ta', 'chef', 'name', 'menu', 'currency', 'currency_for_usd', 'total', 'country', 'best_rev', 'great_rev'], axis=1)
y = df['rating']
X['number_of_reviews'].fillna(X['number_of_reviews'].mean(), inplace=True)
X['new_diff'].fillna(X['new_diff'].min(), inplace=True)
X['price_range'].fillna(0, inplace=True)
X['last_review'].fillna(1, inplace=True)
X.last_.fillna(X['last_'].mean(), inplace=True)
X.reviews_per_people100.fillna(X['reviews_per_people100'].mean(), inplace=True)
#X['total'].fillna(0, inplace=True)
X['restaurants_with_three_stars'].fillna(0, inplace=True)
X['restaurants_with_two_stars'].fillna(0, inplace=True)
X['restaurants_with_one_star'].fillna(0, inplace=True)
X['price_menu_in_usd'].fillna(X['price_menu_in_usd'].mean(), inplace=True)
#X.isnull().sum()


In [19]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [20]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [21]:
# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics  # инструменты для оценки точности модели

In [22]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [23]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.13315205814172043
